# Eigenmode simulation of coupled qubits

In [1]:
%load_ext autoreload
%autoreload 2

## Rendering the design

In [2]:
import names as n
import design as d
from qdesignoptimizer.utils.chip_generation import create_chip_base

design, gui = create_chip_base(n.CHIP_NAME, d.chip_type, open_gui=True)
d.render_qiskit_metal_design(design, gui)

## Creating the study and optimization targets

In [3]:
import mini_studies as ms
import optimization_targets as ot

In [4]:
MINI_STUDY_GROUP = [n.NBR_1, n.NBR_2]
MINI_STUDY = ms.get_mini_study_2qb_resonator_coupler()
RENDER_QISKIT_METAL = lambda design: d.render_qiskit_metal_design(design, gui)

opt_targets = ot.get_opt_targets_2qubits_resonator_coupler(
    groups=MINI_STUDY_GROUP,
    opt_target_qubit_freq=True,
    opt_target_qubit_anharm=True,
    opt_target_resonator_freq=True,
    opt_target_resonator_kappa=False,
    opt_target_resonator_qubit_chi=True,
    opt_target_coupler_freq=True,
    opt_target_coupler_anharm=True,
    opt_target_coupler_qubit_chi=True,
    use_simple_resonator_qubit_chi_relation=False,
)
# attributes = [
#     "qiskit_component_names",
#     "port_list",
#     "open_pins",
#     "modes",
#     "jj_setup",
#     "design_name",
#     "adjustment_rate",
#     "cos_trunc",
#     "fock_trunc",
#     "build_fine_mesh",
#     "nbr_passes",
#     "delta_f"
# ]

# for attr in attributes:
#     print(f"{attr}: {getattr(MINI_STUDY, attr)}")

## Creating design analysis objects

In [5]:
import time
import numpy as np
import parameter_targets as pt
import plot_settings as ps

from qdesignoptimizer.design_analysis import DesignAnalysis, DesignAnalysisState
from qdesignoptimizer.utils.utils import get_save_path

# Closing Ansys hfss
from qdesignoptimizer.utils.utils import close_ansys

close_ansys()

In [6]:
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)

design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path=get_save_path("out/", n.CHIP_NAME),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS_TWO_QB,
)

[INFO|2025-08-23 15:39:05]: self.eig_solver.sim.setup 
{
    "name": "Resonator_setup",
    "reuse_selected_design": true,
    "reuse_setup": true,
    "min_freq_ghz": 1,
    "n_modes": 1,
    "max_delta_f": 0.5,
    "max_passes": 10,
    "min_passes": 1,
    "min_converged": 1,
    "pct_refinement": 30,
    "basis_order": 1,
    "vars": {
        "Lj": "10 nH",
        "Cj": "0 fF"
    }
}
INFO 03:39PM [connect_project]: Connecting to Ansys Desktop API...
INFO 03:39PM [load_ansys_project]: 	Opened Ansys App
INFO 03:39PM [load_ansys_project]: 	Opened Ansys Desktop v2021.1.0
INFO 03:39PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    D:/Ansys/Ansoft/
	Project:   Project466
INFO 03:39PM [connect_design]: No active design found (or error getting active design).
INFO 03:39PM [connect]: 	 Connected to project "Project466". No design detected
03:39PM 15s WARNING [activate_ansys_design]: The design_name=get_mini_study_2qb_resonator_coupler was not in active project.  Designs in activ

## Optimization step

In [7]:
# number of runs of optimization and number of passes for simulation at each run
nbr_iterations = 10
group_passes_start = 17
group_passes_stop = 15
delta_f = 0.001
for i in range(nbr_iterations):
    design_analysis.update_nbr_passes(
        group_passes_start  # min(group_passes_start + i, group_passes_stop)
    )  # successive increase of number of passes for faster convergence [8, 9, 10, 11, 12, 13, 14, 15, 15, 15]
    design_analysis.update_delta_f(delta_f)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

[INFO|2025-08-23 15:39:18]: Updated_design_vars
{
    "cpw_width": "10 um",
    "cpw_gap": "6 um",
    "design_var_width_qubit_1": "180um",
    "design_var_width_qubit_2": "180um",
    "design_var_length_coupler_1to2": "250um",
    "design_var_length_resonator_1_capacitance": "15um",
    "design_var_length_resonator_2_capacitance": "15um",
    "design_var_lj_qubit_1": "10nH",
    "design_var_lj_qubit_2": "12nH",
    "design_var_lj_coupler_1to2": "2.0nH",
    "design_var_cj_qubit_1": "0fF",
    "design_var_cj_qubit_2": "0fF",
    "design_var_cj_coupler_1to2": "0fF",
    "design_var_length_resonator_1": "4200um",
    "design_var_length_resonator_2": "4200um",
    "design_var_coupl_length_qubit_1_resonator_1": "50um",
    "design_var_coupl_length_qubit_2_resonator_2": "50um",
    "design_var_coupl_length_coupler_1to2_qubit_1": "13um",
    "design_var_coupl_length_coupler_1to2_qubit_2": "15um",
    "design_var_coupl_length_resonator_1_tee": "400um",
    "design_var_coupl_length_resonator_2

Initializing new desktop!
PyAEDT INFO: StdOut is enabled
PyAEDT INFO: Log on file is enabled
PyAEDT INFO: Log on Desktop Message Manager is enabled
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT INFO: Launching PyAEDT outside AEDT with CPython and PythonNET.
PyAEDT INFO: AEDT installation Path C:\Program Files\AnsysEM\AnsysEM21.1\Win64.
PyAEDT INFO: Launching AEDT with COM plugin using PythonNET.
PyAEDT WARNING: PyAEDT is not supported in AEDT versions earlier than 2021 R2. Trying to launch PyAEDT with PyWin32.
PyAEDT INFO: pyaedt v0.6.94
PyAEDT INFO: Python version 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:07:43) [MSC v.1942 64 bit (AMD64)]
PyAEDT INFO: AEDT 2021.1.0 Build Date 2020-11-16 21:49:12
PyAEDT INFO: No project is defined. Project Project466 exists and has been read.
PyAEDT INFO: Active Design set to get_mini_study_2qb_resonator_coupler
PyAEDT INFO: Aedt Objects correctly read


INFO 03:39PM [analyze]: Analyzing setup Setup


Design "get_mini_study_2qb_resonator_coupler" info:
	# eigenmodes    5
	# variations    1
Design "get_mini_study_2qb_resonator_coupler" info:
	# eigenmodes    5
	# variations    1
Design "get_mini_study_2qb_resonator_coupler" info:
	# eigenmodes    5
	# variations    1
Design "get_mini_study_2qb_resonator_coupler" info:
	# eigenmodes    5
	# variations    1
Design "get_mini_study_2qb_resonator_coupler" info:
	# eigenmodes    5
	# variations    1

        energy_elec_all       = 3.1989909779825e-25
        energy_elec_substrate = 2.94088118456034e-25
        EPR of substrate = 91.9%

        energy_mag    = 2.8229525690229e-27
        energy_mag % of energy_elec_all  = 0.9%
        

Variation 0  [1/1]

  Mode 0 at 4.57 GHz   [1/5]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               99.1%  1.599e-25 1.411e-27

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_0j   sign s_0j

WARNING 04:06PM [__init__]: <p>Error: <class 'IndexError'></p>


...

INFO 07:59PM [analyze]: Analyzing setup Setup


Design "get_mini_study_2qb_resonator_coupler" info:
	# eigenmodes    5
	# variations    1
Design "get_mini_study_2qb_resonator_coupler" info:
	# eigenmodes    5
	# variations    1
Design "get_mini_study_2qb_resonator_coupler" info:
	# eigenmodes    5
	# variations    1
Design "get_mini_study_2qb_resonator_coupler" info:
	# eigenmodes    5
	# variations    1
Design "get_mini_study_2qb_resonator_coupler" info:
	# eigenmodes    5
	# variations    1

        energy_elec_all       = 2.20631285479638e-25
        energy_elec_substrate = 2.03180264713447e-25
        EPR of substrate = 92.1%

        energy_mag    = 2.20574273773867e-25
        energy_mag % of energy_elec_all  = 100.0%
        

Variation 0  [1/1]

  Mode 0 at 4.25 GHz   [1/5]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               99.5%  7.996e-26 4.057e-28

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_0j   sign 

WARNING 08:29PM [__init__]: <p>Error: <class 'IndexError'></p>



ANALYSIS DONE. Data saved to:

C:\data-pyEPR\Project466\get_mini_study_2qb_resonator_coupler\2025-08-23 20-27-28.npz


	 Differences in variations:



 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Variation 0

C:\Users\marer\Anaconda3\envs\qdesignenv\Lib\site-packages\pyEPR\core_quantum_analysis.py
PJ (5, 3) SJ (5, 3) EJ (3, 3) PHI_zpf (5, 3) PJ_cap (5, 3)
PJ [[2.29593739e-04 9.88788810e-01 5.80782318e-03]
 [9.87206060e-01 4.83063078e-04 6.99842254e-03]
 [7.64475274e-03 5.86400951e-03 9.57708526e-01]
 [1.02677131e-07 1.96523640e-04 1.54804735e-05]
 [2.46019407e-04 1.03796137e-07 1.43887724e-05]]
SJ [[ 1  1  1]
 [-1  1 -1]
 [ 1  1 -1]
 [-1 -1  1]
 [-1 -1  1]]
Om [[4.2485009  0.         0.         0.         0.        ]
 [0.         4.43945639 0.         0.         0.        ]
 [0.         0.         5.55568028 0.         0.        ]
 [0.         0.         0.         7.07193548 0.        ]
 [0.         0.         0.         0.         7.12707205]]
EJ

[INFO|2025-08-23 20:30:40]: freq_ND_results
{
    "0": {
        "0": 3994.850515038946,
        "1": 4171.480159866422,
        "2": 5456.071159634316,
        "3": 7071.8554380076675,
        "4": 7126.971483607424
    }
}
08:30PM 41s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  CRITICAL: QEventDispatcherWin32::wakeUp: Failed to post a message (Not enough quota is available to process this command.)



## Results

### Eigenmodes

In [ ]:
design_analysis.get_eigenmode_results()

### Cross Kerr

In [ ]:
design_analysis.get_cross_kerr_matrix(iteration=-1)

## Update parameters

In [ ]:
design_analysis.overwrite_parameters()

## Close

In [ ]:
from qdesignoptimizer.utils.utils import close_ansys

close_ansys()